# Calculations for *Enhancement in Specific Absorption Rate by Solvent Microencapsulation* by Moore et al 2018
This notebook contains the code used to create all plots in the paper *Enhancement in Specific Absorption Rate by Solvent Microencapsulation*, by Moore et al (2018). Code is written in Julia v0.5.1, and should be legible to users of MATLAB, Python, or other popular scientific computing packages. This code is released in the spirit of scientific transparency, and any errors contained herein are due solely to Thomas Moore.